In [92]:
import pandas as pd
import numpy as np
import json
from opentargets import OpenTargetsClient

In [19]:
ot = OpenTargetsClient()

/home/andrew/miniconda3/envs/variants/lib/python3.8/site-packages/opentargets/conn.py:299: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.api_specs = yaml.load(self.swagger_yaml)


In [69]:
receptors = pd.read_csv('../../receptors/GuideToPharm_GPCR_targets_refined_060121.csv')
receptor_ensembl_ids = receptors['Ensembl Id Grch38']

In [71]:
associations = [list(ot.get_associations_for_target(target_ensembl_id,size=10000)) \
                    for target_ensembl_id in receptor_ensembl_ids]

In [75]:
associations_flattened = [association for association_list in associations for association in association_list]

In [81]:
with open('../data/opentargets_associations_raw.json', 'w') as fout:
    json.dump(associations_flattened, fout)

In [85]:
associations_flattened[0]

{'target': {'tractability': {'other_modalities': {'buckets': [],
    'categories': {'clinical_precedence': 0.0}},
   'smallmolecule': {'top_category': 'Predicted_Tractable_sm',
    'buckets': [8],
    'high_quality_compounds': 0,
    'small_molecule_genome_member': True,
    'categories': {'clinical_precedence': 0.0,
     'predicted_tractable': 0.3,
     'discovery_precedence': 0.0}},
   'antibody': {'top_category': 'Predicted_Tractable_ab_High_confidence',
    'buckets': [5, 7],
    'categories': {'predicted_tractable_med_low_confidence': 0.25,
     'clinical_precedence': 0.0,
     'predicted_tractable_high_confidence': 0.3}}},
  'gene_info': {'symbol': 'ACKR1',
   'name': 'atypical chemokine receptor 1 (Duffy blood group)'},
  'id': 'ENSG00000213088'},
 'association_score': {'datatypes': {'literature': 0.0,
   'rna_expression': 0.0,
   'genetic_association': 1.0,
   'somatic_mutation': 0.0,
   'known_drug': 0.0,
   'animal_model': 0.0,
   'affected_pathway': 0.0},
  'overall': 1.0,
 

In [175]:
def flatten_opentargets_association(x):
    output = {
        'gene_symbol':x['target']['gene_info']['symbol'],
        'gene_ensembl_id':x['target']['id'],
        'gene_name':x['target']['gene_info']['name'],
        'disease_id':x['disease']['id'],
        'disease_type':x['disease']['efo_info']['therapeutic_area']['labels'],
        'disease_name':x['disease']['efo_info']['label'],
        'association_id':x['id'],
        'overall_association_score':x['association_score']['overall'],
        'association_scores': x['association_score']['datatypes'],
    }

    return output

flatten_opentargets_association(associations_flattened[0])

{'gene_symbol': 'ACKR1',
 'gene_ensembl_id': 'ENSG00000213088',
 'gene_name': 'atypical chemokine receptor 1 (Duffy blood group)',
 'disease_id': 'EFO_0009390',
 'disease_type': ['measurement'],
 'disease_name': 'sum of neutrophil and eosinophil counts',
 'association_id': 'ENSG00000213088-EFO_0009390',
 'overall_association_score': 1.0,
 'association_scores': {'literature': 0.0,
  'rna_expression': 0.0,
  'genetic_association': 1.0,
  'somatic_mutation': 0.0,
  'known_drug': 0.0,
  'animal_model': 0.0,
  'affected_pathway': 0.0}}

In [176]:
associations_tidy = [flatten_opentargets_association(association) for association in associations_flattened]

In [177]:
associations_df = pd.DataFrame(associations_tidy)

In [178]:
association_with_scores = pd.concat(
    (
        associations_df.drop('association_scores',axis=1),
        pd.json_normalize(associations_df['association_scores'])
    ), 
    axis=1
)
association_with_scores

,gene_symbol,gene_ensembl_id,gene_name,disease_id,disease_type,disease_name,association_id,overall_association_score,literature,rna_expression,genetic_association,somatic_mutation,known_drug,animal_model,affected_pathway
0,ACKR1,ENSG00000213088,atypical chemokine receptor 1 (Duffy blood group),EFO_0009390,[measurement],sum of neutrophil and eosinophil counts,ENSG00000213088-EFO_0009390,1.00000,0.0000,0.00000,1.0,0.0,0.0,0.0,0.0
1,ACKR1,ENSG00000213088,atypical chemokine receptor 1 (Duffy blood group),EFO_0009388,[measurement],sum of basophil and neutrophil counts,ENSG00000213088-EFO_0009388,1.00000,0.0000,0.00000,1.0,0.0,0.0,0.0,0.0
2,ACKR1,ENSG00000213088,atypical chemokine receptor 1 (Duffy blood group),EFO_0007993,[measurement],lymphocyte percentage of leukocytes,ENSG00000213088-EFO_0007993,1.00000,0.0000,0.00000,1.0,0.0,0.0,0.0,0.0
3,ACKR1,ENSG00000213088,atypical chemokine receptor 1 (Duffy blood group),EFO_0007992,[measurement],basophil percentage of leukocytes,ENSG00000213088-EFO_0007992,1.00000,0.0000,0.00000,1.0,0.0,0.0,0.0,0.0
4,ACKR1,ENSG00000213088,atypical chemokine receptor 1 (Duffy blood group),EFO_0007990,[measurement],neutrophil percentage of leukocytes,ENSG00000213088-EFO_0007990,1.00000,0.0000,0.00000,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201322,XCR1,ENSG00000173578,X-C motif chemokine receptor 1,MONDO_0004938,[psychiatric disorder],substance dependence,ENSG00000173578-MONDO_0004938,0.00680,0.0068,0.00000,0.0,0.0,0.0,0.0,0.0
201323,XCR1,ENSG00000173578,X-C motif chemokine receptor 1,MONDO_0002494,[psychiatric disorder],substance-related disorder,ENSG00000173578-MONDO_0002494,0.00680,0.0068,0.00000,0.0,0.0,0.0,0.0,0.0
201324,XCR1,ENSG00000173578,X-C motif chemokine receptor 1,EFO_0001062,[infectious disease],cytomegalovirus infection,ENSG00000173578-EFO_0001062,0.00680,0.0068,0.00000,0.0,0.0,0.0,0.0,0.0
201325,XCR1,ENSG00000173578,X-C motif chemokine receptor 1,EFO_0008510,"[infectious disease, nervous system disease]",Lyme disease,ENSG00000173578-EFO_0008510,0.00678,0.0000,0.00678,0.0,0.0,0.0,0.0,0.0


In [179]:
association_with_scores.to_csv('../data/opentargets_associations.csv')

In [182]:
def extract_tractability(x):
    output = {
        'gene_symbol':x['target']['gene_info']['symbol'],
        'gene_ensembl_id':x['target']['id'],
        'gene_name':x['target']['gene_info']['name'],
    }
    if 'tractability' in x['target'].keys():
        output['small_molecule_tractability_category'] = x['target']['tractability']['smallmolecule']['top_category']
        output['antibody_tractability_category'] = x['target']['tractability']['antibody']['top_category']
    return output

extract_tractability(associations_flattened[0])

{'gene_symbol': 'ACKR1',
 'gene_ensembl_id': 'ENSG00000213088',
 'gene_name': 'atypical chemokine receptor 1 (Duffy blood group)',
 'small_molecule_tractability_category': 'Predicted_Tractable_sm',
 'antibody_tractability_category': 'Predicted_Tractable_ab_High_confidence'}

In [185]:
tractability_df = pd.DataFrame([extract_tractability(association) for association in associations_flattened])
tractability_df = tractability_df.drop_duplicates().reset_index().drop('index',axis=1)
tractability_df

,gene_symbol,gene_ensembl_id,gene_name,small_molecule_tractability_category,antibody_tractability_category
0,ACKR1,ENSG00000213088,atypical chemokine receptor 1 (Duffy blood group),Predicted_Tractable_sm,Predicted_Tractable_ab_High_confidence
1,ACKR2,ENSG00000144648,atypical chemokine receptor 2,Predicted_Tractable_sm,Predicted_Tractable_ab_High_confidence
2,ACKR3,ENSG00000144476,atypical chemokine receptor 3,Discovery_Precedence_sm,Predicted_Tractable_ab_High_confidence
3,ACKR4,ENSG00000129048,atypical chemokine receptor 4,Predicted_Tractable_sm,Predicted_Tractable_ab_High_confidence
4,ADCYAP1R1,ENSG00000078549,ADCYAP receptor type I,Predicted_Tractable_sm,Predicted_Tractable_ab_High_confidence
...,...,...,...,...,...
388,TSHR,ENSG00000165409,thyroid stimulating hormone receptor,Predicted_Tractable_sm,Predicted_Tractable_ab_High_confidence
389,UTS2R,ENSG00000181408,urotensin 2 receptor,Clinical_Precedence_sm,Predicted_Tractable_ab_High_confidence
390,VIPR1,ENSG00000114812,vasoactive intestinal peptide receptor 1,Unknown,Predicted_Tractable_ab_High_confidence
391,VIPR2,ENSG00000106018,vasoactive intestinal peptide receptor 2,Discovery_Precedence_sm,Predicted_Tractable_ab_High_confidence


In [186]:
tractability_df.to_csv('../data/opentargets_tractability.csv')